In [34]:
import pandas as pd
import requests

from datetime import datetime
import json
import time

In [88]:
def request_wrapper(API_address, printing=False):
    if printing:
        print('API request: ' + API_address)
    fail_count = 0
    response = requests.get(API_address)
    while response.status_code != 200:
        print("API request ERROR: ", requests.status_codes._codes[response.status_code][0], API_address)
        fail_count += 1
        if fail_count >= 5:
            raise SystemExit("ERROR: API request failed 5 times in a row.")
        if response.status_code == 429:
            wait_time = float(response.headers['Retry-After'])
            print("WARNING: Rate Limit Exceeded...retrying request after {} seconds".format(wait_time))
            time.sleep(wait_time)
        response = requests.get(API_address)
    return response

In [61]:
# Enter your own API key here
# my API Key, Expires: Sat, May 30th, 2020 @ 9:53am (PT) in 23 hours and 59 minutes
API_key = "RGAPI-e55ca1a4-b7e9-46d7-93ab-ea38ff3c47ba"
API_key_suffix = '?api_key=' + API_key

regions = ['br1', 'eun1', 'euw1', 'jp1', 'kr', 'la1', 'la2', 'na1', 'oc1', 'tr1', 'ru']
regions_dict = {'br1':'americas',
                'eun1':'europe',
                'euw1':'europe',
                'jp1':'asia',
                'kr':'asia',
                'la1':'americas',
                'la2':'americas',
                'na1':'americas',
                'oc1':'americas',
                'tr1':'europe',
                'ru':'europe'
               }

In [81]:
#Specify league, region and patch
league = 'challenger'
region = 'euw1'
patch = '10.11'



In [80]:
# Get summonerId
API_address_league = 'https://' + region + '.api.riotgames.com/tft/league/v1/' + league

response = request_wrapper(API_address_league + API_key_suffix)
summonerId_list = [i['summonerId'] for i in response.json()['entries']]

In [62]:
# Get puuid for each summonerId
API_address_summoner = 'https://' + region + '.api.riotgames.com/tft/summoner/v1/summoners/'

puuid_list = []
for summonerId in summonerId_list:
    response = request_wrapper(API_address_summoner + summonerId + API_key_suffix)
    puuid_list.append(response.json()['puuid'])

In [78]:
# Get matchId for each puuid
# Really annoying since you can't check which patch they were played on until the next step
# Change count to change number of matches, change this depending on how long the patch has been out
n_matches = 20
API_address_matchId = 'https://' + regions_dict[region] + '.api.riotgames.com/tft/match/v1/matches/by-puuid/'

matchId_list = []
for puuid in puuid_list:
    response = request_wrapper(API_address_matchId + puuid + '/ids?count={}&api_key='.format(n_matches) + API_key)
    matchId_list += response.json()
matchId_list = list(set(matchId_list))

API request ERROR:  too_many_requests
API request ERROR:  too_many_requests


In [98]:
# Get match data for each matchId
API_address_match = 'https://' + regions_dict[region] + '.api.riotgames.com/tft/match/v1/matches/'

match_data_json = []
for matchId in matchId_list:
    response = request_wrapper(API_address_match + matchId + API_key_suffix)
    match_data_json.append(response.json())

API request ERROR:  too_many_requests https://europe.api.riotgames.com/tft/match/v1/matches/EUW1_4615313660?api_key=RGAPI-e55ca1a4-b7e9-46d7-93ab-ea38ff3c47ba
API request ERROR:  too_many_requests https://europe.api.riotgames.com/tft/match/v1/matches/EUW1_4612793642?api_key=RGAPI-e55ca1a4-b7e9-46d7-93ab-ea38ff3c47ba
API request ERROR:  too_many_requests https://europe.api.riotgames.com/tft/match/v1/matches/EUW1_4622954639?api_key=RGAPI-e55ca1a4-b7e9-46d7-93ab-ea38ff3c47ba
API request ERROR:  too_many_requests https://europe.api.riotgames.com/tft/match/v1/matches/EUW1_4630300206?api_key=RGAPI-e55ca1a4-b7e9-46d7-93ab-ea38ff3c47ba
API request ERROR:  too_many_requests https://europe.api.riotgames.com/tft/match/v1/matches/EUW1_4586054830?api_key=RGAPI-e55ca1a4-b7e9-46d7-93ab-ea38ff3c47ba
API request ERROR:  too_many_requests https://europe.api.riotgames.com/tft/match/v1/matches/EUW1_4616049905?api_key=RGAPI-e55ca1a4-b7e9-46d7-93ab-ea38ff3c47ba
API request ERROR:  too_many_requests https://

In [128]:
# Filter match data by patch
def get_patch(match_data):
    return '.'.join(match_data['info']['game_version'].split(' ', 1)[1].split('.', 2)[0:2])

match_data_json = list(filter(lambda x: get_patch(x) == patch, match_data_json))


In [131]:
# Save data
with open('match_data/match_data_challenger_euw_10.11_300520.txt', 'w') as outfile:
    json.dump(match_data_json, outfile)
    
    